# Canonical Correlations for positions in space

In [1]:
#import useful libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pickle
import configuration
import general_statistics as stats
import figures as figs
import matplotlib.cm as cm
from matplotlib import colors
from scipy import signal
from scipy import stats as sstats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy
cmap = cm.jet
import math
from sklearn.datasets import load_iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.cross_decomposition import CCA
from sklearn.model_selection import cross_val_score
import random
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")

In [2]:
# define mouse information, and preprocessin versions from analysis.
# load data for all sessions in this mouse and compute mean activity for each neuron.

mouse = 32363       ### mouse number id
decoding_v = 1            ## decoding version, normaly equal to one
motion_correction_v = 20  ### 100 means everything was aligned, 200 means it was also equalized, 20 is for day wise
alignment_v = 3           ## alignment version, version 3 is for day wise
equalization_v = 0        ## equalization version
source_extraction_v = 1   ## source extraction version
component_evaluation_v = 1 ## component evaluation version

# here we will do analysis with no registration
# registration_v = 2        ## registration version
sf = 10                   ## sampling frequency of the original signal 
re_sf= 1                 ## value of resampling
session_now = 2
BINS = 70
period = int(5 * sf / re_sf)
period_resting = period *50
fixed = 'None'
## define task for plotting. This will cahnge for other mice!!!!

if mouse == 56165 or mouse == 32364:
    if mouse == 56165:
        sessions = [1,2,4]       ## sessions for this particular mouse
    if mouse == 32364:
        sessions = [1,2]
    if session_now == 1:
        task = 'OVERLAPPING'
        colapse_behaviour = 1
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        if mouse == 32364:
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   
        if mouse == 56165:
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]   

    else:
        if session_now == 2:
            task = 'STABLE'
            colapse_behaviour = 2
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
        else:
            task = 'RANDOM'
            colapse_behaviour = 0
            labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32365:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 2:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 3:
            task = 'OVERLAPPING'
            fixed = 'LR'
            object_fixed = 4
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]           
            
if mouse == 56166:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]

    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UR'
            object_fixed = 5
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32366:
    sessions = [2,3] ## sessions for this particular mouse
    if session_now == 3:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
            
if mouse == 32363:
    sessions = [1,2] ## sessions for this particular mouse
    if session_now == 1:
        task = 'RANDOM'
        colapse_behaviour = 0
        labels =['Unlabel','Rest1', 'Navigation', 'Obj1' , 'Obj2', 'Run1', 'Run2']
        colornames=['k',"r","deepskyblue","g","blue","g","blue"]
    else:
        if session_now == 2:
            task = 'OVERLAPPING'
            fixed = 'UL'
            object_fixed = 6
            colapse_behaviour = 1
            labels =['Unlabel','Rest1', 'Navigation', 'Overlap_object' , 'Moving_object','RunOO' , 'RunMO' ]
            colornames=['k',"r","deepskyblue","g","blue","g","blue"]

In [3]:
file_directory = os.environ['PROJECT_DIR'] + 'neural_analysis/data/calcium_activity_day_wise/'
timeline_file_dir = os.environ['PROJECT_DIR'] + 'neural_analysis/data/timeline/'
behaviour_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
behaviour_dir_parameters = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram_parameters/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
tracking_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/center_of_mass/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
objects_dir= os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/object_positions/'
figure_path = os.environ['PROJECT_DIR'] +'neural_analysis/figures/trial_aligned_events/'

In [4]:
# define all relevant files names and load them into a list
session = session_now

activity_list = []
timeline_list = []
behaviour_list = []
behaviour_list_copy = []
behaviour_list_unsup = []
parameters_list = []
tracking_list = []
total_time = 0
day = 0
print('LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR')
for trial in [1,6,11,16]:

    beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_ethogram.npy'
    ## LOAD BEHAVIOUR
    behaviour = np.load(behaviour_dir + beh_file_name_1)
    reshape_behaviour = np.reshape(behaviour[:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
    resample_beh1 = np.reshape(scipy.stats.mode(reshape_behaviour,axis=1)[0],reshape_behaviour.shape[0])

    beh_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75_parameters.npy'
    ## LOAD PARAMETRS FOR BEHAVIOUR CLASSIFICATION
    parameters = np.load(behaviour_dir_parameters + beh_file_name_1)
    params = []
    for param in range(parameters.shape[0]):
        r1_params = np.reshape(parameters[param,:int(int(behaviour.shape[0]/re_sf)*re_sf)],(int(behaviour.shape[0]/re_sf),re_sf))
        r2_params = np.reshape(scipy.stats.mode(r1_params,axis=1)[0],reshape_behaviour.shape[0])
        params.append(r2_params)
    resample_params = np.array(params)
        
    ## LOAD TRACKING
    tracking_file_name_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_day_' + f'{day+1}' + '_likelihood_0.75.npy'
    position = np.load(tracking_dir + tracking_file_name_1)
    resample_position, resample_position_stc = stats.resample_matrix(neural_activity=position.T,
                                                                                            re_sf=re_sf)
    ## LOAD TIMELINE
    time_file_session_1 =  'mouse_'+ f'{mouse}'+'_session_'+ f'{session}' +'_trial_'+ f'{trial}'+'_v1.3.1.0_10.pkl'

    timeline_file= open(timeline_file_dir + time_file_session_1,'rb')
    timeline_info = pickle.load(timeline_file)
    timeline_1 = np.zeros(len(timeline_info) + 1)
    for i in range(len(timeline_info)):
        timeline_1[i] = timeline_info[i][1]
    timeline_1[len(timeline_info)] = behaviour.shape[0]
    timeline = timeline_1/re_sf
    time_lenght = 10
    resample_timeline = timeline_1/re_sf
    timeline_list.append(resample_timeline)

    behaviour_list.append(resample_beh1)
    behaviour_list_copy.append(resample_beh1.copy())
    parameters_list.append(resample_params)
    tracking_list.append(resample_position)
    total_time = total_time + behaviour.shape[0]


    file_name_session_1 = 'mouse_' + f'{mouse}' + '_session_' + f'{session}' + '_trial_'+ f'{trial}'+'_v' + f'{decoding_v}' + '.4.' + f'{motion_correction_v}' + \
                            '.' + f'{alignment_v}' + '.' + f'{equalization_v}' + '.' + f'{source_extraction_v}' + '.' + \
                          f'{component_evaluation_v}' +  '.0.npy'

     ##load activity and timeline
    activity = np.load(file_directory + file_name_session_1)
    neural_activity1 = activity[1:,:]
    ## z-score neural activity
    neural_activity = neural_activity1
    ##downsample neural activity
    resample_neural_activity_mean, resample_neural_activity_std = stats.resample_matrix(neural_activity=neural_activity,
                                                                                            re_sf=re_sf)

    activity_list.append(resample_neural_activity_mean)

    day = day + 1
    print(day)

        
id_target = [0,1,2,3,4,5,6] # 0: unlabeled, 1:resting, 2:Navigation, 3: Obj1, 4:Obj2, 5:Run1, 6:Run2

if colapse_behaviour == 0 : # RANDOM
    for day in range(len(behaviour_list)):
        for trial in range(5):
            behaviour_trial = behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])]
            objects = np.unique(behaviour_trial)
            selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
            index0 = np.where(behaviour_trial==objects[selected_object])[0]
            index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-4], behaviour_trial!=objects[selected_object]))
            index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-3], behaviour_trial!=objects[selected_object]))
            behaviour_trial[index0] = 3
            behaviour_trial[index1] = 4
            behaviour_trial[index2] = 4            
            
            index0 = np.where(behaviour_trial==objects[selected_object]+4)[0]
            index1 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-2], behaviour_trial!=objects[selected_object]+4))
            index2 = np.where(np.logical_and(behaviour_trial==objects[len(objects)-1], behaviour_trial!=objects[selected_object]+4))
            behaviour_trial[index0] = 5
            behaviour_trial[index1] = 6
            behaviour_trial[index2] = 6 
            
            behaviour_list[day][int(timeline_list[day][2*trial]):int(timeline_list[day][2*trial+1])] = behaviour_trial


if colapse_behaviour == 1 : #OVERLAPPING
    for day in range(len(behaviour_list)):
        behaviour_list[day][np.where(behaviour_list[day] == object_fixed)[0]] = 100
        behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=3, behaviour_list[day]<=6))[0]] = 4
        behaviour_list[day][np.where(behaviour_list[day] == 100)[0]] = 3        
        behaviour_list[day][np.where(behaviour_list[day] == object_fixed +4)[0]] = 200        
        behaviour_list[day][np.where(np.logical_and(behaviour_list[day]>=7, behaviour_list[day]<=10))[0]] = 6
        behaviour_list[day][np.where(behaviour_list[day] == 200)[0]] = 5        


if colapse_behaviour == 2: #STABLE
    for day in range(len(behaviour_list)):
        objects = np.unique(behaviour_list[day])
        selected_object = np.random.randint(len(objects)-4,len(objects)-2,1)
        index0 = np.where(behaviour_list[day]==objects[selected_object])[0]
        index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-4], behaviour_list[day]!=objects[selected_object]))
        index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-3], behaviour_list[day]!=objects[selected_object]))
        behaviour_list[day][index0] = 3
        behaviour_list[day][index1] = 4
        behaviour_list[day][index2] = 4      
    
        index0 = np.where(behaviour_list[day]==objects[selected_object]+4)[0]
        index1 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-2], behaviour_list[day]!=objects[selected_object]+4))
        index2 = np.where(np.logical_and(behaviour_list[day]==objects[len(objects)-1], behaviour_list[day]!=objects[selected_object]+4))
        behaviour_list[day][index0] = 5
        behaviour_list[day][index1] = 6
        behaviour_list[day][index2] = 6  

LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR
1
2
3
4


In [5]:
from scipy.ndimage import gaussian_filter

def occupancy_map(tracking = None, binnumber = BINS):
    
    x = np.array([int(tracking[0,i]) for i in range(tracking.shape[1])])
    y = np.array([int(tracking[1,i]) for i in range(tracking.shape[1])])
    xpos = x[np.logical_and(x>0,y>0)]
    ypos = y[np.logical_and(x>0,y>0)]
    
    occupancy = np.zeros((binnumber,binnumber))
    xdenom = int(np.max(xpos)/binnumber+1)
    ydenom = int(np.max(ypos)/binnumber+1)
    data_points = 0
    for x,y in zip(xpos,ypos):
        if x>0 and y>0:
            x1 = int(x/xdenom)
            y1 = int(y/ydenom)
            occupancy [x1,y1]= occupancy[x1,y1] + 1
            data_points+=1
    occupancy = occupancy/data_points
    return occupancy

def binned_tracking(tracking = None, binnumber = 70):
    
    x = np.array([int(tracking[0,i]) for i in range(tracking.shape[1])])
    y = np.array([int(tracking[1,i]) for i in range(tracking.shape[1])])
    
    xpos = x[np.logical_and(x>0,y>0)]
    ypos = y[np.logical_and(x>0,y>0)]
    
    xdenom = int(np.max(xpos)/binnumber+1)
    ydenom = int(np.max(ypos)/binnumber+1)
    
    xnew = [int(xpos[i]/xdenom) for i in range(len(xpos))]
    ynew = [int(ypos[i]/xdenom) for i in range(len(xpos))]
    
    return xnew,ynew

def neuron_activity_map(activity= None,tracking = None,binnumber = 70, neuronID = 0):
     
    x = np.array([int(tracking[0,i]) for i in range(tracking.shape[1])])
    y = np.array([int(tracking[1,i]) for i in range(tracking.shape[1])])
    xpos = x[np.logical_and(x>0,y>0)]
    ypos = y[np.logical_and(x>0,y>0)]
    
    act_map = np.zeros((binnumber,binnumber))
    xdenom = int(np.max(xpos)/binnumber+1)
    ydenom = int(np.max(ypos)/binnumber+1)
    counter = 0
    for x,y in zip(xpos,ypos):
        if x>0 and y>0:
            x1 = int(x/xdenom)
            y1 = int(y/ydenom)
            act_map[x1,y1]+=np.mean(activity[neuronID,counter])
            counter+=1
        
    act_map = act_map/counter
    return act_map

def mean_activity_map(activity= None,tracking = None,binnumber = BINS):
    
    x = np.array([int(tracking[0,i]) for i in range(tracking.shape[1])])
    y = np.array([int(tracking[1,i]) for i in range(tracking.shape[1])])
    xpos = x[np.logical_and(x>0,y>0)]
    ypos = y[np.logical_and(x>0,y>0)]
    act_map = np.zeros((binnumber,binnumber))
    xdenom = int(np.max(xpos)/binnumber+1)
    ydenom = int(np.max(ypos)/binnumber+1)
    counter = 0
    for x,y in zip(xpos,ypos):
        if x>0 and y>0:
            x1 = int(x/xdenom)
            y1 = int(y/ydenom)
            act_map[x1,y1]+=np.mean(activity[:,counter])
            counter+=1
        
    act_map = act_map/counter
    return act_map

def speed_selection(tracking = None, speed_limit = 3):
    
    x = tracking[0,:]
    y = tracking[1,:]
    vx = np.diff(x)
    vy = np.diff(y)
    speed = np.sqrt(vx*vx+vy*vy)
    index = np.where(speed > speed_limit)[0]
    
    return index

def poisson_pdf(n,l):
    return pow(l,n)*exp(-l)/math.factorial(n)

from scipy.ndimage import gaussian_filter
sigma = 1.5

In [6]:
pca = PCA()
activity_list_pca = []
variance_list = []
variance_ratio_list = []

#cca_components = min(13,activity_list[0].shape[0])
#cca = CCA(n_components=cca_components)
activity_list_cca = []

for day in range(len(behaviour_list_copy)):
    ### run pca on the entire dataset
    cca_components = min(13,activity_list[day].shape[0])
    cca = CCA(n_components=cca_components)
    pca.fit(activity_list[day].T)
    transformed_activity = pca.fit(activity_list[day].T).transform(activity_list[day].T)
    activity_list_pca.append(transformed_activity.T)
    variance_list.append(pca.explained_variance_/(1+np.sqrt(activity_list[day].shape[0]/activity_list[day].shape[1]))**2)
    normalized_variance = pca.explained_variance_/(1+np.sqrt(activity_list[day].shape[0]/activity_list[day].shape[1]))**2
    variance_ratio_list.append(np.cumsum(normalized_variance/sum(normalized_variance)))
    activity_list_cca.append(cca.fit(activity_list[day].T, parameters_list[day].T).transform(activity_list[day].T))
    


In [7]:
    
    #for each day creates a list that counts and saves times of different events.
    events_position_day_list = []
    events_position_counter_day_list = []
    events_position_time_starts_day = []
    for day in range(len(behaviour_list_copy)):
        events_list = []
        events_counter_list = []
        events_time_starts = []
        random_events = []
        start_counter = 100
        counter = 0
        for i in range(behaviour_list_copy[day].shape[0]):
            if behaviour_list_copy[day][i] != start_counter:
                events_list.append(start_counter)
                events_counter_list.append(counter)
                events_time_starts.append(i)
                start_counter = behaviour_list_copy[day][i]
                counter = 1
            else:
                counter = counter + 1      
        events_position_day_list.append(events_list)
        events_position_counter_day_list.append(events_counter_list)
        events_position_time_starts_day.append(events_time_starts)
 

    id_target = [0,1,2,3,4,5,6,7,8,9,10]
    ## put all events together and take neural activity from each event
    events_position_activity_pre_norm= []
    events_position_duration_list = []
    events_position_activity_pca = []
    events_position_activity_cca = []

    for day in range(len(behaviour_list_copy)):
        target_activity = []
        target_activity_pca = []
        target_activity_cca = []
        events_duration_day = []
        for target in id_target:
            events = np.array(events_position_day_list[day])
            events_counter = np.array(events_position_counter_day_list[day])
            events_time = np.array(events_position_time_starts_day[day])

            position_events = np.where(events == target)[0]
            events_duration = events_counter[position_events]   # convert to seconds
            time = events_time[position_events]
            i = 0
            event_target = []
            event_target_pca = []
            event_target_cca = []
            events_duration_target = np.zeros(len(events_duration),)
            for event in events_duration:
                if event > 10:
                    local_activity = activity_list[day][:,time[i]-period:time[i]+period]
                    local_activity_pca = activity_list_pca[day][:,time[i]-period:time[i]+period]
                    local_activity_cca = activity_list_cca[day][time[i]-period:time[i]+period,:].T
                    event_target.append(local_activity)
                    event_target_pca.append(local_activity_pca)
                    event_target_cca.append(local_activity_cca)
                    events_duration_target[i]=1
                i = i + 1
            target_activity.append(event_target)
            target_activity_pca.append(event_target_pca)
            target_activity_cca.append(event_target_cca)
            events_duration_day.append(events_duration_target)
        events_position_activity_pre_norm.append(target_activity)
        events_position_duration_list.append(events_duration_day)
        events_position_activity_pca.append(target_activity_pca)
        events_position_activity_cca.append(target_activity_cca)
    events_position_activity = events_position_activity_pre_norm

    # for each day creates a list that counts and saves times of different events.
    events_day_list = []
    events_counter_day_list = []
    events_time_starts_day = []
    for day in range(len(behaviour_list)):
        events_list = []
        events_counter_list = []
        events_time_starts = []
        random_events = []
        start_counter = 100
        counter = 0
        for i in range(behaviour_list[day].shape[0]):
            if behaviour_list[day][i] != start_counter:
                events_list.append(start_counter)
                events_counter_list.append(counter)
                events_time_starts.append(i)
                start_counter = behaviour_list[day][i]
                counter = 1
            else:
                counter = counter + 1      
        events_day_list.append(events_list)
        events_counter_day_list.append(events_counter_list)
        events_time_starts_day.append(events_time_starts)
        
    ## put all events together and take neural activity from each event
    events_activity_pre_norm= []
    events_duration_list = []
    events_activity_pca = []
    events_activity_cca = []


    for day in range(len(behaviour_list)):
        target_activity = []
        target_activity_pca = []
        target_activity_cca = []
        events_duration_day = []
        for target in id_target:
            events = np.array(events_day_list[day])
            events_counter = np.array(events_counter_day_list[day])
            events_time = np.array(events_time_starts_day[day])

            position_events = np.where(events == target)[0]
            events_duration = events_counter[position_events]   # convert to seconds
            time = events_time[position_events]
            i = 0
            event_target = []
            event_target_pca = []
            event_target_cca = []
            events_duration_target = np.zeros(len(events_duration),)
            for event in events_duration:
                if event > 10:
                    local_activity = activity_list[day][:,time[i]-period:time[i]+period]
                    local_activity_pca = activity_list_pca[day][:,time[i]-period:time[i]+period]
                    local_activity_cca = activity_list_cca[day][time[i]-period:time[i]+period,:].T
                    event_target.append(local_activity)
                    event_target_pca.append(local_activity_pca)
                    event_target_cca.append(local_activity_cca)
                    events_duration_target[i]=1
                i = i + 1
            target_activity.append(event_target)
            target_activity_pca.append(event_target_pca)
            target_activity_cca.append(event_target_cca)
            events_duration_day.append(events_duration_target)
        events_activity_pre_norm.append(target_activity)
        events_duration_list.append(events_duration_day)
        events_activity_pca.append(target_activity_pca)
        events_activity_cca.append(target_activity_cca)
    events_activity = events_activity_pre_norm
 

In [ ]:
#from matplotlib.collections import LineCollection





In [8]:
       
for day in range(4):
    targets_in_day = np.unique(behaviour_list_copy[day])
    print(targets_in_day)
    selection_target = targets_in_day[np.logical_and(targets_in_day>=3,targets_in_day<=6)]
    selection_target = np.array([int(selection_target[i]) for i in range(len(selection_target))])
    print(selection_target)

    behaviour_label = ['x','x','x','LL','LR','UR','UL']
    behaviour = []
    for j in range(selection_target.shape[0]):
        behaviour.append(behaviour_label[selection_target[j]])
    
    print(behaviour)

    pca = PCA()

    figure = plt.figure()
    gs = plt.GridSpec(60, 30)
    
    color_code = ['Reds','Blues','Greens','Oranges']
    color_code_pop = ['r','b','g','orange']
    j=0
    
    ### definition of subplots that have plots from different exploratory behaviour    
    axes_pc_trajectory1 = figure.add_subplot(gs[19:28,0:7],projection = '3d')
    axes_pc_trajectory1.set_title('Trajectories in PC Space', fontsize = 20)
    axes_pc_trajectory1.set_xlabel('1PC',fontsize = 15)
    axes_pc_trajectory1.set_ylabel('2PC',fontsize = 15)
    axes_pc_trajectory1.set_zlabel('3PC',fontsize = 15)
    
    pc_axes_min = np.zeros((6,3))
    pc_axes_max = np.zeros((6,3))
    cc_axes_min = np.zeros((6,3))
    cc_axes_max = np.zeros((6,3))

    axes_cc_trajectory1 = figure.add_subplot(gs[19:28,10:17],projection = '3d')
    axes_cc_trajectory1.set_title('Trajectories in CC Space', fontsize = 20)
    axes_cc_trajectory1.set_xlabel('1CC',fontsize = 15)
    axes_cc_trajectory1.set_ylabel('2CC',fontsize = 15)
    axes_cc_trajectory1.set_zlabel('3CC',fontsize = 15)

    
    for target in selection_target:
        
        ### subplots for mean activity 
        axes = figure.add_subplot(gs[0:12, j])
        axes_pca_space = figure.add_subplot(gs[0:12, j+7])
        axes_cca_space = figure.add_subplot(gs[0:12, j+14])

        trial_activity = np.zeros(( events_position_activity[day][target][0].shape[0], events_position_activity[day][target][0].shape[1]))
        trial_activity_complete = np.zeros(( events_position_activity[day][target][0].shape[0],len( events_position_activity[day][target])*10+1))
        trial_activity_all_data = np.zeros(( events_position_activity[day][target][0].shape[0],len( events_position_activity[day][target])*2*period+1))
        trial_activity_pca = np.zeros(( events_position_activity[day][target][0].shape[0], events_position_activity[day][target][0].shape[1]))
        trial_activity_cca = np.zeros((cca_components, events_position_activity[day][target][0].shape[1]))
       
        ### generate matrix with mean activity and entire trial repetitions activity
        for neuron in range( events_position_activity[day][target][0].shape[0]):
            neuron_trial_activity = np.zeros(( events_position_activity[day][target][0].shape[1],))
            component_trial_activity = np.zeros(( events_position_activity[day][target][0].shape[1],))
            cca_trial_activity = np.zeros(( events_position_activity[day][target][0].shape[1],))
            init = 0
            init2=0
            for trial in range(len(events_position_activity[day][target])):
                neuron_trial_activity +=  events_position_activity[day][target][trial][neuron,:]#/(np.max(events_activity[day][target][trial][neuron,:])-np.min(events_activity[day][target][trial][neuron,:]))
                component_trial_activity +=events_position_activity_pca[day][target][trial][neuron,:]
                if neuron < cca_components:
                    cca_trial_activity +=events_position_activity_cca[day][target][trial][neuron,:]
                trial_activity_all_data[neuron,init2:init2+2*period]= events_position_activity[day][target][trial][neuron,:]
                trial_activity_complete[neuron,init:init+10] = events_position_activity[day][target][trial][neuron,period:period+10]
                init = init+10
                init2=init2+2*period
            neuron_trial_activity = neuron_trial_activity / len(events_position_activity[day][target])
            component_trial_activity = component_trial_activity / len(events_position_activity[day][target])
            trial_activity[neuron,:] = neuron_trial_activity
            trial_activity_pca[neuron,:] = component_trial_activity
            if neuron < cca_components:
                trial_activity_cca[neuron,:] = cca_trial_activity / cca_components

        time_axis = np.arange(-period,period)/10
        neurons_axis = np.arange(0,trial_activity.shape[0])        
        mesh = axes.pcolormesh(time_axis,neurons_axis, trial_activity, cmap=color_code[j],shading='auto')
        axes.set_xlabel('Time [s]',fontsize = 20)
        axes.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes.set_ylabel('Neurons', fontsize = 20)
        axes.set_title(behaviour[j], fontsize = 20)
        axes.vlines(0,0, neuron, color = color_code_pop[j])
        
        axes_1 = figure.add_subplot(gs[14:16,j])
        pstd = np.mean(trial_activity,axis = 0)
        pstd_std_error = np.std(trial_activity,axis = 0)/np.sqrt(trial_activity.shape[0])
        uperbound = pstd + pstd_std_error
        lowerbound = pstd - pstd_std_error
        axes_1.plot(time_axis, pstd,'k', linewidth = 2)
        axes_1.fill_between(time_axis , lowerbound , uperbound)#, edgecolor=color[i], facecolor = color[i])
        axes_1.set_title('PSTH', fontsize = 15)
        axes_1.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_1.set_ylabel('Mean Activity [a.u.]',fontsize = 15)
        axes_1.set_xticks([])
        axes_1.set_xlabel('Time [s]',fontsize = 15)

        axes_pca_space.axes.pcolormesh(time_axis,neurons_axis, trial_activity_pca, cmap=color_code[j],shading='auto')
        axes_pca_space.set_xlabel('Time [s]',fontsize = 20)
    
        axes_cca_space.axes.pcolormesh(time_axis,neurons_axis[0:cca_components], trial_activity_cca, cmap=color_code[j],shading='auto')
        axes_cca_space.set_xlabel('Time [s]',fontsize = 20)
    
        color = np.linspace(0, 20, 2*period)
        axes_pc_trajectory1.scatter(trial_activity_pca[0,:],trial_activity_pca[1,:],trial_activity_pca[2,:], c = color, cmap = color_code[j])
        axes_cc_trajectory1.scatter(trial_activity_cca[0,:],trial_activity_cca[1,:],trial_activity_cca[2,:], c = color, cmap = color_code[j])
        
        for dim in range(3):
            pc_axes_min[j,dim] = np.min(trial_activity_pca[dim,:])
            pc_axes_max[j,dim] = np.max(trial_activity_pca[dim,:])
            cc_axes_min[j,dim] = np.min(trial_activity_cca[dim,:])
            cc_axes_max[j,dim] = np.max(trial_activity_cca[dim,:])      
        
        #axes.set_yticks()
        axes_pca_space.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes_pca_space.set_ylabel('PC components', fontsize = 20)
        axes_pca_space.set_title( behaviour[j], fontsize = 20)
        axes_pca_space.vlines(0,0,trial_activity_pca.shape[0], color = color_code_pop[j])
        
        #axes.set_yticks()
        axes_cca_space.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes_cca_space.set_ylabel('CC components', fontsize = 20)
        axes_cca_space.set_title( behaviour[j], fontsize = 20)
        axes_cca_space.vlines(0,0, trial_activity_pca.shape[0], color = color_code_pop[j])
        
        axes_2 = figure.add_subplot(gs[14:16,j+7])
        pstd = np.mean(trial_activity_pca,axis = 0)
        pstd_std_error = np.std(trial_activity_pca,axis = 0)/np.sqrt(trial_activity_pca.shape[0])
        uperbound = pstd + pstd_std_error
        lowerbound = pstd - pstd_std_error
        axes_2.plot(time_axis, pstd,'k', linewidth = 2)
        axes_2.fill_between(time_axis , lowerbound , uperbound)#, edgecolor=color[i], facecolor = color[i])
        axes_2.set_title('PCA PSTH', fontsize = 15)
        axes_2.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_2.set_ylabel('Mean Activity [a.u.]',fontsize = 15)
        axes_2.set_xticks([])
        axes_2.set_xlabel('Time [s]',fontsize = 15)
        
        axes_3 = figure.add_subplot(gs[14:16,j+14])
        pstd = trial_activity_cca[0,:]
        axes_3.plot(time_axis, pstd,'k', linewidth = 2)
        axes_3.set_title('1st CC', fontsize = 15)
        axes_3.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_1.set_ylabel('Trasformed Activity [a.u.]',fontsize = 15)
        axes_3.set_xticks([])
        axes_3.set_xlabel('Time [s]',fontsize = 15)
           
        j=j+1
    
    axes_pc_trajectory = figure.add_subplot(gs[49:58,0:7],projection = '3d')
    axes_pc_trajectory.set_title('Trajectories in PC Space', fontsize = 20)
    axes_pc_trajectory.set_xlabel('1PC',fontsize = 15)
    axes_pc_trajectory.set_ylabel('2PC',fontsize = 15)
    axes_pc_trajectory.set_zlabel('3PC',fontsize = 15)
    
    axes_cc_trajectory = figure.add_subplot(gs[49:58,10:17],projection = '3d')
    axes_cc_trajectory.set_title('Trajectories in CC Space', fontsize = 20)
    axes_cc_trajectory.set_xlabel('1CC',fontsize = 15)
    axes_cc_trajectory.set_ylabel('2CC',fontsize = 15)
    axes_cc_trajectory.set_zlabel('3CC',fontsize = 15)
    

    behaviour = ['Object1','Object2']    
    j=0
    for target in [3,4]:
        
        ### subplots for mean activity 
        axes = figure.add_subplot(gs[30:42, 3*j:3*j+3])
        axes_pca_space = figure.add_subplot(gs[30:42, 3*j+7:3*j+10])
        axes_cca_space = figure.add_subplot(gs[30:42, 3*j+14:3*j+17])

        
        trial_activity = np.zeros((events_activity[day][target][0].shape[0],events_activity[day][target][0].shape[1]))
        trial_activity_complete = np.zeros((events_activity[day][target][0].shape[0],len(events_activity[day][target])*10+1))
        trial_activity_all_data = np.zeros((events_activity[day][target][0].shape[0],len(events_activity[day][target])*2*period+1))
        trial_activity_pca = np.zeros((events_activity[day][target][0].shape[0],events_activity[day][target][0].shape[1]))
        trial_activity_cca = np.zeros((cca_components,events_activity[day][target][0].shape[1]))
       
        ### generate matrix with mean activity and entire trial repetitions activity
        for neuron in range(events_activity[day][target][0].shape[0]):
            neuron_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
            component_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
            cca_trial_activity = np.zeros((events_activity[day][target][0].shape[1],))
            init = 0
            init2=0
            for trial in range(len(events_activity[day][target])):
                neuron_trial_activity += events_activity[day][target][trial][neuron,:]#/(np.max(events_activity[day][target][trial][neuron,:])-np.min(events_activity[day][target][trial][neuron,:]))
                component_trial_activity +=events_activity_pca[day][target][trial][neuron,:]
                if neuron < cca_components:
                    cca_trial_activity +=events_activity_cca[day][target][trial][neuron,:]
                trial_activity_all_data[neuron,init2:init2+2*period]= events_activity[day][target][trial][neuron,:]
                trial_activity_complete[neuron,init:init+10] = events_activity[day][target][trial][neuron,period:period+10]
                init = init+10
                init2=init2+2*period
            neuron_trial_activity = neuron_trial_activity / len(events_activity[day][target])
            component_trial_activity = component_trial_activity / len(events_activity[day][target])
            trial_activity[neuron,:] = neuron_trial_activity
            trial_activity_pca[neuron,:] = component_trial_activity
            if neuron < cca_components:
                trial_activity_cca[neuron,:] = cca_trial_activity / cca_components
        
        time_axis = np.arange(-period,period)/10
        neurons_axis = np.arange(0,trial_activity.shape[0])        
        mesh = axes.pcolormesh(time_axis,neurons_axis, trial_activity, cmap=color_code[j],shading='auto')
        axes.set_xlabel('Time [s]',fontsize = 20)
        axes.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes.set_ylabel('Neurons', fontsize = 20)
        axes.set_title('Behaviour:' + behaviour[j], fontsize = 20)
        axes.vlines(0,0, neuron, color = color_code_pop[j])
        
        axes_1 = figure.add_subplot(gs[44:46,3*j:3*j+3])
        pstd = np.mean(trial_activity,axis = 0)
        pstd_std_error = np.std(trial_activity,axis = 0)/np.sqrt(trial_activity.shape[0])
        uperbound = pstd + pstd_std_error
        lowerbound = pstd - pstd_std_error
        axes_1.plot(time_axis, pstd,'k', linewidth = 2)
        axes_1.fill_between(time_axis , lowerbound , uperbound)#, edgecolor=color[i], facecolor = color[i])
        axes_1.set_title('PSTH', fontsize = 15)
        axes_1.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_1.set_ylabel('Mean Activity [a.u.]',fontsize = 15)
        axes_1.set_xticks([])
        axes_1.set_xlabel('Time [s]',fontsize = 15)

        axes_pca_space.axes.pcolormesh(time_axis,neurons_axis, trial_activity_pca, cmap=color_code[j],shading='auto')
        axes_pca_space.set_xlabel('Time [s]',fontsize = 20)
    
        axes_cca_space.axes.pcolormesh(time_axis,neurons_axis[0:cca_components], trial_activity_cca, cmap=color_code[j],shading='auto')
        axes_cca_space.set_xlabel('Time [s]',fontsize = 20)
    
        color = np.linspace(0, 20, 2*period)
        axes_pc_trajectory.scatter(trial_activity_pca[0,:],trial_activity_pca[1,:],trial_activity_pca[2,:], c = color, cmap = color_code[j])
        axes_cc_trajectory.scatter(trial_activity_cca[0,:],trial_activity_cca[1,:],trial_activity_cca[2,:], c = color, cmap = color_code[j])
        
        for dim in range(3):
            pc_axes_min[j+4,dim] = np.min(trial_activity_pca[dim,:])
            pc_axes_max[j+4,dim] = np.max(trial_activity_pca[dim,:])
            
            cc_axes_min[j+4,dim] = np.min(trial_activity_cca[dim,:])
            cc_axes_max[j+4,dim] = np.max(trial_activity_cca[dim,:])

        #axes.set_yticks()
        axes_pca_space.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes_pca_space.set_ylabel('PC components', fontsize = 20)
        axes_pca_space.set_title('Behaviour:' + behaviour[j], fontsize = 20)
        axes_pca_space.vlines(0,0, trial_activity_pca.shape[0], color = color_code_pop[j])
        
        #axes.set_yticks()
        axes_cca_space.set_xlabel('Time [s]', fontsize = 20)
        if j==0:
            axes_cca_space.set_ylabel('CC components', fontsize = 20)
        axes_cca_space.set_title('Behaviour:' + behaviour[j], fontsize = 20)
        axes_cca_space.vlines(0,0, trial_activity_pca.shape[0], color = color_code_pop[j])
        
        axes_2 = figure.add_subplot(gs[44:46,3*j+7:3*j+10])
        pstd = np.mean(trial_activity_pca,axis = 0)
        pstd_std_error = np.std(trial_activity_pca,axis = 0)/np.sqrt(trial_activity_pca.shape[0])
        uperbound = pstd + pstd_std_error
        lowerbound = pstd - pstd_std_error
        axes_2.plot(time_axis, pstd,'k', linewidth = 2)
        axes_2.fill_between(time_axis , lowerbound , uperbound)#, edgecolor=color[i], facecolor = color[i])
        axes_2.set_title('PCA PSTH', fontsize = 15)
        axes_2.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_2.set_ylabel('Mean Activity [a.u.]',fontsize = 15)
        axes_2.set_xticks([])
        axes_2.set_xlabel('Time [s]',fontsize = 15)
        
        axes_3 = figure.add_subplot(gs[44:46,3*j+14:3*j+17])
        pstd = trial_activity_cca[0,:]
        axes_3.plot(time_axis, pstd,'k', linewidth = 2)
        axes_3.set_title('1st CC', fontsize = 15)
        axes_3.vlines(0,min(pstd), max(pstd),color = 'k',linestyle = '-')
        if j==0:
            axes_1.set_ylabel('Trasformed Activity [a.u.]',fontsize = 15)
        axes_3.set_xticks([])
        axes_3.set_xlabel('Time [s]',fontsize = 15)
           
        j=j+1
    
    pc_ax_min = np.min(pc_axes_min,axis=0)
    pc_ax_max = np.max(pc_axes_max,axis=0)

    cc_ax_min = np.min(cc_axes_min,axis=0)
    cc_ax_max = np.max(cc_axes_max,axis=0)


    axes_pc_trajectory1.set_xlim([pc_ax_min[0],pc_ax_max[0]])
    axes_pc_trajectory1.set_ylim([pc_ax_min[1],pc_ax_max[1]])
    axes_pc_trajectory1.set_zlim([pc_ax_min[2],pc_ax_max[2]])
    
    axes_cc_trajectory1.set_xlim([cc_ax_min[0],cc_ax_max[0]])
    axes_cc_trajectory1.set_ylim([cc_ax_min[1],cc_ax_max[1]])
    axes_cc_trajectory1.set_zlim([cc_ax_min[2],cc_ax_max[2]])

    axes_pc_trajectory.set_xlim([pc_ax_min[0],pc_ax_max[0]])
    axes_pc_trajectory.set_ylim([pc_ax_min[1],pc_ax_max[1]])
    axes_pc_trajectory.set_zlim([pc_ax_min[2],pc_ax_max[2]])
    
    axes_cc_trajectory.set_xlim([cc_ax_min[0],cc_ax_max[0]])
    axes_cc_trajectory.set_ylim([cc_ax_min[1],cc_ax_max[1]])
    axes_cc_trajectory.set_zlim([cc_ax_min[2],cc_ax_max[2]])

    
    figure.set_size_inches([50., 40])

    figure.suptitle(' Traces events plot ' + task +' TRAJECTORIES_'+ ' fixed = ' + fixed   , fontsize = 20)
    figure_name = figure_path + 'traces_events_corners_'+f'{mouse}'+'_session_'+f'{session}'+\
                                 '_day_'+f'{day}'+'_binsize_'+f'{re_sf}'+'_period_'+ f'{period}'  +'.png'
    figure.savefig(figure_name)
    #plt.close()
    
 

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[3 4 5 6]
['LL', 'LR', 'UR', 'UL']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[3 4 5 6]
['LL', 'LR', 'UR', 'UL']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[3 4 5 6]
['LL', 'LR', 'UR', 'UL']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[3 4 5 6]
['LL', 'LR', 'UR', 'UL']
